**Test Notebook**

In [2]:
%run /config_All_Zones

In [3]:
display(dbutils.fs.ls("dbfs:/mnt/silver/"))

path,name,size
dbfs:/mnt/silver/_$azuretmpfolder$/,_$azuretmpfolder$/,0
dbfs:/mnt/silver/cab/,cab/,0
dbfs:/mnt/silver/dynamics/,dynamics/,0


In [4]:
display(dbutils.fs.ls("dbfs:/mnt/silver/dynamics/"))

path,name,size
dbfs:/mnt/silver/dynamics/audits/,audits/,0
dbfs:/mnt/silver/dynamics/incidents/,incidents/,0
dbfs:/mnt/silver/dynamics/opportunity/,opportunity/,0
dbfs:/mnt/silver/dynamics/string_map/,string_map/,0


In [5]:
display(dbutils.fs.ls("dbfs:/mnt/silver/cab/"))


path,name,size
dbfs:/mnt/silver/cab/accounts/,accounts/,0
dbfs:/mnt/silver/cab/companies/,companies/,0
dbfs:/mnt/silver/cab/company_features/,company_features/,0
dbfs:/mnt/silver/cab/company_modules/,company_modules/,0
dbfs:/mnt/silver/cab/devices/,devices/,0
dbfs:/mnt/silver/cab/form_headers/,form_headers/,0
dbfs:/mnt/silver/cab/geofences/,geofences/,0
dbfs:/mnt/silver/cab/licenses/,licenses/,0
dbfs:/mnt/silver/cab/modules/,modules/,0
dbfs:/mnt/silver/cab/order_headers/,order_headers/,0


In [6]:
companies = spark.read.format('delta').load("/mnt/silver/cab/companies")
accounts = spark.read.format('delta').load("/mnt/silver/cab/accounts")
licenses = spark.read.format('delta').load("/mnt/silver/cab/licenses")
trips = spark.read.format('delta').load("/mnt/silver/cab/trips")
form_headers = spark.read.format('delta').load("/mnt/silver/cab/form_headers")
incidents = spark.read.format('delta').load("/mnt/silver/dynamics/incidents")
timekeeping_statuses = spark.read.format('delta').load("/mnt/silver/cab/timekeeping_statuses")
order_headers = spark.read.format('delta').load("/mnt/silver/cab/order_headers")


In [7]:
from pyspark.sql.functions import sum, min, max, col, current_date, udf
from pyspark.sql.types import *

In [8]:
companies.count()

Out[18]: 21890

In [9]:
# AccountId=2520 has multiple companies
display(companies.filter('AccountId=4700'))

# companies.filter('SetupCompletionTime is null').count()

CompanyId,InstanceId,InstanceName,AccountId,AccountName,TierId,TierName,Status,IsDeleted,IsActive,BillingCreatedUtc,IsPending,NonCab,VerticalId,VerticalName,OperationalCreatedUtc,DataPurgeCompleted,CompanyName,ContactName,Phone,ContactEmail,AddressLine2,AddressLine1,City,RegionName,PostalCode,CountryCode,SubRegionName,RegionCode,SubRegionCode,UserIntegrationType,Market,EnableCustomerDW,SetupCompletionTime,Modified,SilverModifiedUtc
5788,50,AT&T,4700,Connect,10,Standard,Active,false,true,2017-02-22T21:32:32.610+0000,false,false,4,Field Service,2017-02-22T21:34:05.750+0000,false,Connect,Sener Degirmenci,(480) 627-2201,republic@vwpcs.com,null,18500 N Allied Way,Phoenix,AZ,85054,US,null,US-AZ,null,0,null,null,2017-02-22T21:34:05.750+0000,2018-10-16T19:23:17.690+0000,2020-08-14T07:04:21.740+0000


In [10]:
display(accounts.filter('BillingAccountId='))

accountid,accountnumber,modifiedon,industrycode,BillingAccountId,Culture,AccountName,Status,IsTest,DataPurgedCompleted,CreationDate,ModificationDate,OldAccountId
be03e9c2-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:06:45.000+0000,null,27,en-US,QA-ATT,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
b394efc8-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:19:10.000+0000,null,44,en-US,Val's Std TestAccount,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
0f7e05d5-7377-e611-80f2-00155d002f2e,null,2020-06-13T08:57:04.000+0000,null,53,en-US,JW ATT 7 (nonbillable),1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
29e901cf-7377-e611-80f2-00155d002f2e,null,2020-06-13T08:57:00.000+0000,null,65,en-US,JW ATT 11 (MGR),1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
247e05d5-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:21:13.000+0000,null,76,en-US,William R ATT Production,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
5ee10ddb-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:03:48.000+0000,null,81,en-US,"Nullpointer, LLC",1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
91b708e1-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:13:38.000+0000,null,103,en-US,TekMark1,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
af350be7-7377-e611-80f2-00155d002f2e,null,2020-06-13T09:13:39.000+0000,null,108,en-US,TekSecure Labs,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
d4d217f3-7377-e611-80f2-00155d002f2e,null,2020-06-24T17:30:21.000+0000,null,115,en-US,ABC Construction,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null
f7d217f3-7377-e611-80f2-00155d002f2e,null,2020-06-13T08:38:15.000+0000,null,126,en-US,AT&T ATO Testing,1,true,null,2015-07-22T16:43:35.990+0000,2020-05-06T03:14:15.040+0000,null


In [11]:
# display(licenses)
# active companyId = 13 with license changes
# display(licenses.select('LicenseId', 'AccountId', 'CompanyId', 'InstanceId', 'Status', 'Count', 'ActivationReasonName', 'OriginalLicenseId', 'ProductDisplayName').filter('CompanyId=13'))

# repulic (accountId 4700 or companyId 5788) with lots of licenses.
display(licenses.filter('CompanyId=5788 and status=0'))


LicenseId,AccountId,CompanyId,InstanceId,Status,StatusName,TierId,TierName,SOCID,SOC,SOCDescription,MRC,Count,PTN,CreatedOn,ExpirationDate,PaymentScheduleId,PaymentMethodId,BillingDay,BillingFrequencyUnit,NextChargeDate,PaymentModificationDate,BillingFrequencyInterval,PaymentCreationDate,PaymentMethod,BillingProviderId,BillingProvider,BillingId,PaymentMethodExpirationDate,PaymentMethodStatus,CustomerId,LicenseEngaged,Billable,BillingIntegration,Category,ConfirmedState,ConfirmedStateName,DeactivationReason,DeactivationReasonName,DeactDate,ModificationDate,ProductPriceLevelId,ProductId,ProductDisplayName,Standalone,OriginalLicenseId,OriginalActivationDate,ActivationReason,ActivationReasonName,SilverModifiedUtc
148849,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,16104769112,2018-01-05T00:00:00.000+0000,null,12085,14173,8,3,2020-08-08,2020-07-08T01:19:52.610+0000,1,2017-09-08T15:47:25.670+0000,ATT 287250815930,2,ATT,287250815930,null,1,04982323,true,true,3,0,0,Active,null,null,null,2018-02-13T17:31:24.340+0000,1001,1001,GPS Add-on,false,null,2018-01-05T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
148868,4700,5788,50,0,Active,10,Standard,21,ATTWFMS,User License Standard,null,1,14845001025,2018-01-05T00:00:00.000+0000,null,12085,14173,8,3,2020-08-08,2020-07-08T01:19:52.610+0000,1,2017-09-08T15:47:25.670+0000,ATT 287250815930,2,ATT,287250815930,null,1,04982323,true,true,3,0,0,Active,null,null,null,2018-02-13T17:31:24.340+0000,1,1,User License,true,null,2018-01-05T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
148986,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,19712161988,2018-01-05T00:00:00.000+0000,null,11776,13770,25,3,2020-07-25,2020-06-25T01:18:21.990+0000,1,2017-08-25T21:33:24.050+0000,ATT 287253447730,2,ATT,287253447730,null,1,05264693,true,true,3,0,0,Active,null,null,null,2018-02-13T17:30:08.230+0000,1001,1001,GPS Add-on,false,null,2018-01-05T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
150366,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,18125644993,2018-01-11T00:00:00.000+0000,null,14080,16616,5,3,2020-08-05,2020-07-05T01:20:17.260+0000,1,2018-01-05T14:51:46.770+0000,ATT 287253448225,2,ATT,287253448225,null,1,05264832,true,true,3,0,0,Active,null,null,null,2018-02-13T17:24:19.360+0000,1001,1001,GPS Add-on,false,null,2018-01-11T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
150699,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,12096238471,2018-01-13T00:00:00.000+0000,null,11776,13770,25,3,2020-07-25,2020-06-25T01:18:21.990+0000,1,2017-08-25T21:33:24.050+0000,ATT 287253447730,2,ATT,287253447730,null,1,05264693,true,true,3,0,0,Active,null,null,null,2018-05-30T13:16:26.860+0000,1001,1001,GPS Add-on,false,null,2018-01-13T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
150712,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,12095619129,2018-01-13T00:00:00.000+0000,null,11776,13770,25,3,2020-07-25,2020-06-25T01:18:21.990+0000,1,2017-08-25T21:33:24.050+0000,ATT 287253447730,2,ATT,287253447730,null,1,05264693,true,true,3,0,0,Active,null,null,null,2018-02-13T17:30:08.230+0000,1001,1001,GPS Add-on,false,null,2018-01-13T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
150715,4700,5788,50,0,Active,10,Standard,21,ATTWFMS,User License Standard,null,1,12096235389,2018-01-13T00:00:00.000+0000,null,11776,13770,25,3,2020-07-25,2020-06-25T01:18:21.990+0000,1,2017-08-25T21:33:24.050+0000,ATT 287253447730,2,ATT,287253447730,null,1,05264693,true,true,3,0,0,Active,null,null,null,2018-02-13T17:30:08.230+0000,1,1,User License,true,null,2018-01-13T00:00:00.000+0000,8,CarrierNotification,2020-08-14T08:05:55.382+0000
150752,4700,5788,50,0,Active,10,Standard,24,ATTWFMADD,GPS Add-on,null,1,15735338914,2018-01-13T00:00:00.000+0000,null,14080,16616,5,3,2020-08-05,2020-07-05T01:20:17.260+0000,1,2018-01-05T14:51:46.770+0000,ATT 287253448

In [12]:
# churned CompanyId=23 with license changes
display(licenses.select('LicenseId', 'AccountId', 'CompanyId', 'InstanceId', 'Status', 'Count', 'ActivationReasonName', 'OriginalLicenseId', 'ProductDisplayName', 'CreatedOn', 'DeactDate') \
        .filter('Status=1 and CompanyId=23'))

# display(licenses.filter('OriginalLicenseId is not null'))

LicenseId,AccountId,CompanyId,InstanceId,Status,Count,ActivationReasonName,OriginalLicenseId,ProductDisplayName,CreatedOn,DeactDate
278135,14490,23,57,1,5,LicenseChangeRequest,null,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:16:19.310+0000
278162,14490,23,57,1,2,LicenseChangeRequest,null,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:21.420+0000
278165,14490,23,57,1,2,AddPromo,null,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:18:02.650+0000
278159,14490,23,57,1,1,LicenseChangeRequest,null,GPS Add-on,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:51.690+0000
278164,14490,23,57,1,2,ChangeTier,278162,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:59.460+0000
278160,14490,23,57,1,2,LicenseChangeRequest,null,Vehicles,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:42.850+0000
278161,14490,23,57,1,1,AddPromo,null,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:21.230+0000
278163,14490,23,57,1,1,ChangeTier,278161,User License,2019-03-18T00:00:00.000+0000,2019-03-18T13:17:25.800+0000
278166,14490,23,57,1,1,LicenseChangeRequest,null,GPS Add-on,2019-03-18T00:00:00.000+0000,2019-03-18T13:19:33.980+0000


In [13]:
display(incidents)

statecode,statename,accountid,modifiedon,statuscode,statusname,incidentid,casetypecode,casetype,prioritycode,priority,severitycode,severity,createdon,resolvedon,companyid,isescalated
1,Resolved,89f81254-115c-e411-80e9-00155d007a09,2014-10-25T18:16:01.000+0000,5,Problem Solved,000387f7-725c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2008-09-25T12:24:23.000+0000,2014-10-25T18:16:01.000+0000,null,false
1,Resolved,64d95cdb-f2d5-e311-80d2-00155d002f03,2014-10-26T00:00:30.000+0000,5,Problem Solved,00044313-a35c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2012-09-10T13:25:38.000+0000,2014-10-26T00:00:30.000+0000,null,false
1,Resolved,aa2e928e-425c-e411-80e9-00155d007a09,2014-10-27T23:36:28.000+0000,5,Problem Solved,0007d621-615c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2008-01-17T16:22:19.000+0000,2014-10-27T23:36:28.000+0000,null,false
1,Resolved,f0ae8dbd-375c-e411-80e9-00155d007a09,2014-10-27T22:26:01.000+0000,5,Problem Solved,0009042e-955c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2006-12-04T20:44:04.000+0000,2014-10-27T22:26:01.000+0000,null,false
1,Resolved,7286e5b7-f932-e411-80e3-00155d007a09,2017-03-27T12:46:55.000+0000,5,Problem Solved,000be92d-eb12-e711-80f9-00155d002f2e,1,Question,2,Normal,100000002,Normal,2017-03-27T12:44:56.000+0000,2017-03-27T12:46:55.000+0000,null,false
1,Resolved,62d95cdb-f2d5-e311-80d2-00155d002f03,2016-12-07T16:43:59.000+0000,5,Problem Solved,000f6481-ab94-e611-80f4-00155d002f2e,3,Request,2,Normal,100000002,Normal,2016-10-17T20:51:47.000+0000,2016-10-17T20:57:23.000+0000,null,false
1,Resolved,bc09fd26-255c-e411-80e9-00155d007a09,2014-10-27T21:05:40.000+0000,5,Problem Solved,0010381c-745c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2005-09-19T22:17:48.000+0000,2014-10-27T21:05:40.000+0000,null,false
1,Resolved,56e3a446-1b5c-e411-80e9-00155d007a09,2014-10-25T17:15:50.000+0000,5,Problem Solved,0011678d-6a5c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2011-02-17T16:41:25.000+0000,2014-10-25T17:15:50.000+0000,null,false
1,Resolved,fddc82a7-1b5c-e411-80e9-00155d007a09,2014-10-27T22:40:45.000+0000,5,Problem Solved,00156184-9b5c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2007-03-01T18:45:02.000+0000,2014-10-27T22:40:45.000+0000,null,false
1,Resolved,68fc6a49-ce58-e511-8116-00155d007a09,2017-05-08T17:26:23.000+0000,5,Problem Solved,0017a554-1334-e711-80fb-00155d007a25,2,Problem,2,Normal,100000002,Normal,2017-05-08T17:25:29.000+0000,2017-05-08T17:26:23.000+0000,null,false


In [14]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import tree
from subprocess import call

In [15]:
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(df[data.feature_names], df['target'], random_state=0)

In [17]:

# Step 1: Import the model you want to use
# This was already imported earlier in the notebook so commenting out
#from sklearn.tree import DecisionTreeClassifier
# Step 2: Make an instance of the Model
clf = DecisionTreeClassifier(max_depth = 2, 
                             random_state = 0)
# Step 3: Train the model on the data
clf.fit(X_train, Y_train)
# Step 4: Predict labels of unseen (test) data
# Not doing this step in the tutorial
# clf.predict(X_test)

Out[4]: DecisionTreeClassifier(max_depth=2, random_state=0)

In [18]:

# def plot_graphviz_databricks(model, filename="tree.dot", png="tree.png"):
#   fn=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
#   cn=['setosa', 'versicolor', 'virginica']

#   tree.export_graphviz(model,
#                        out_file=filename,
#                        feature_names = fn, 
#                        class_names=cn,
#                        filled = True)
  
#   call(['dot', '-Tpng', filename, '-o', 'png', '-Gdpi=600'])
  
#   plt.figure(figsize = (14, 18))
#   plt.imshow(plt.imread(png))
#   plt.axis('off');
#   plt.show();

In [19]:
# plot_graphviz_databricks(clf)

In [20]:
# from IPython.display import Image
# import graphviz 
# import pydotplus
# from sklearn.externals.six import StringIO

# fn=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
# cn=['setosa', 'versicolor', 'virginica']

# dot_data = StringIO()
# tree.export_graphviz(clf, out_file=dot_data, filled=True)  

# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# graph.write_png('tree.png')

In [21]:
# plot xgboost tree
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from xgboost import plot_tree, to_graphviz
import graphviz

# load data
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X_train, X_test, Y_train, Y_test = train_test_split(df[data.feature_names], df['target'], random_state=0)

# fit model no training data
model = XGBClassifier()
model.fit(X_train, Y_train)

# plot single tree
# plot_tree(model)
# plt.show()

dot = to_graphviz(model)  
dot.render("tree", view=True)

--------------------------------------------------------------------------- 
 PermissionError Traceback (most recent call last)
 <command-481533886879559> in <module> 
 26 
 27 dot = to_graphviz ( model ) 
 ---> 28 dot . render ( "tree" , view = True ) 

 /databricks/python/lib/python3.7/site-packages/graphviz/files.py in render (self, filename, directory, view, cleanup, format, renderer, formatter, quiet, quiet_view) 
 200 relative to the DOT source file . 
 201 """
 --> 202 filepath = self . save ( filename , directory ) 
 203 
 204 if format is None : 

 /databricks/python/lib/python3.7/site-packages/graphviz/files.py in save (self, filename, directory) 
 164 
 165 log . debug ( 'write %d bytes to %r' , len ( data ) , filepath ) 
 --> 166 with io . open ( filepath , 'w' , encoding = self . encoding ) as fd : 
 167 fd . write ( data ) 
 168 if not data . endswith ( u'\n' ) : 

 PermissionError : [Errno 13] Permission denied: 'tree'

In [22]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from joblibspark import register_spark
from sklearn.utils import parallel_backend

register_spark() # register spark backend

iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = svm.SVC(gamma='auto')

clf = GridSearchCV(svr, parameters, cv=5)

with parallel_backend('spark', n_jobs=3):
    clf.fit(iris.data, iris.target)

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-4101097221978692> in <module> 
 13 
 14 with parallel_backend ( 'spark' , n_jobs = 3 ) : 
 ---> 15 clf . fit ( iris . data , iris . target ) 

 /databricks/python/lib/python3.7/site-packages/sklearn/utils/validation.py in inner_f (*args, **kwargs) 
 70 FutureWarning)
 71 kwargs . update ( { k : arg for k , arg in zip ( sig . parameters , args ) } ) 
 ---> 72 return f ( ** kwargs ) 
 73 return inner_f
 74 

 /databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_search.py in fit (self, X, y, groups, **fit_params) 
 693 verbose=self.verbose)
 694 results = { } 
 --> 695 with parallel : 
 696 all_candidate_params = [ ] 
 697 all_out = [ ] 

 /databricks/python/lib/python3.7/site-packages/joblib/parallel.py in __enter__ (self) 
 698 def __enter__ ( self ) : 
 699 self . _managed_backend = True 
 --> 700 self . _initialize_backend ( ) 
 701 return self
 702 

 /databricks/python/lib/python3.7/site-packages/joblib/parallel.py in _initialize_backend (self) 
 709 try : 
 710 n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
 --> 711 **self._backend_args)
 712 if self . timeout is not None and not self . _backend . supports_timeout : 
 713 warnings.warn(

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in configure (self, n_jobs, parallel, prefer, require, **backend_args) 
 108 
 109 def configure ( self , n_jobs = 1 , parallel = None , prefer = None , require = None , ** backend_args ) : 
 --> 110 n_jobs = self . effective_n_jobs ( n_jobs ) 
 111 self . _n_jobs = n_jobs
 112 return n_jobs

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in effective_n_jobs (self, n_jobs) 
 79 
 80 def effective_n_jobs ( self , n_jobs ) : 
 ---> 81 max_num_concurrent_tasks = self . _get_max_num_concurrent_tasks ( ) 
 82 if n_jobs is None : 
 83 n_jobs = 1 

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in _get_max_num_concurrent_tasks (self) 
 96 # maxNumConcurrentTasks() is a package private API 
 97 # pylint: disable=W0212 
 ---> 98 return self . _spark . sparkContext . _jsc . sc ( ) . maxNumConcurrentTasks ( ) 
 99 
 100 def abort_everything ( self , ensure_ready = True ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 330 raise Py4JError(
 331 "An error occurred while calling {0}{1}{2}. Trace:\n{3}\n" . 
 --> 332 format(target_id, ".", name, value))
 333 else : 
 334 raise Py4JError(

 Py4JError : An error occurred while calling o261.maxNumConcurrentTasks. Trace:
py4j.security.Py4JSecurityException: Method public int org.apache.spark.SparkContext.maxNumConcurrentTasks() is not whitelisted on class class org.apache.spark.SparkContext
	at py4j.security.WhitelistingPy4JSecurityManager.checkCall(WhitelistingPy4JSecurityManager.java:435)
	at py4j.Gateway.invoke(Gateway.java:294)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [23]:
from sklearn.utils import parallel_backend
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
from joblibspark import register_spark

register_spark() # register spark backend

iris = datasets.load_iris()
clf = svm.SVC(kernel='linear', C=1)
with parallel_backend('spark', n_jobs=3):
  scores = cross_val_score(clf, iris.data, iris.target, cv=5)

print(scores)

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-4101097221978697> in <module> 
 10 clf = svm . SVC ( kernel = 'linear' , C = 1 ) 
 11 with parallel_backend ( 'spark' , n_jobs = 3 ) : 
 ---> 12 scores = cross_val_score ( clf , iris . data , iris . target , cv = 5 ) 
 13 
 14 print ( scores ) 

 /databricks/python/lib/python3.7/site-packages/sklearn/utils/validation.py in inner_f (*args, **kwargs) 
 70 FutureWarning)
 71 kwargs . update ( { k : arg for k , arg in zip ( sig . parameters , args ) } ) 
 ---> 72 return f ( ** kwargs ) 
 73 return inner_f
 74 

 /databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py in cross_val_score (estimator, X, y, groups, scoring, cv, n_jobs, verbose, fit_params, pre_dispatch, error_score) 
 404 fit_params = fit_params , 
 405 pre_dispatch = pre_dispatch , 
 --> 406 error_score=error_score)
 407 return cv_results [ 'test_score' ] 
 408 

 /databricks/python/lib/python3.7/site-packages/sklearn/utils/validation.py in inner_f (*args, **kwargs) 
 70 FutureWarning)
 71 kwargs . update ( { k : arg for k , arg in zip ( sig . parameters , args ) } ) 
 ---> 72 return f ( ** kwargs ) 
 73 return inner_f
 74 

 /databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py in cross_validate (estimator, X, y, groups, scoring, cv, n_jobs, verbose, fit_params, pre_dispatch, return_train_score, return_estimator, error_score) 
 246 return_times = True , return_estimator = return_estimator , 
 247 error_score=error_score)
 --> 248 for train, test in cv.split(X, y, groups))
 249 
 250 zipped_scores = list ( zip ( * scores ) ) 

 /databricks/python/lib/python3.7/site-packages/joblib/parallel.py in __call__ (self, iterable) 
 948 
 949 if not self . _managed_backend : 
 --> 950 n_jobs = self . _initialize_backend ( ) 
 951 else : 
 952 n_jobs = self . _effective_n_jobs ( ) 

 /databricks/python/lib/python3.7/site-packages/joblib/parallel.py in _initialize_backend (self) 
 709 try : 
 710 n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
 --> 711 **self._backend_args)
 712 if self . timeout is not None and not self . _backend . supports_timeout : 
 713 warnings.warn(

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in configure (self, n_jobs, parallel, prefer, require, **backend_args) 
 108 
 109 def configure ( self , n_jobs = 1 , parallel = None , prefer = None , require = None , ** backend_args ) : 
 --> 110 n_jobs = self . effective_n_jobs ( n_jobs ) 
 111 self . _n_jobs = n_jobs
 112 return n_jobs

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in effective_n_jobs (self, n_jobs) 
 79 
 80 def effective_n_jobs ( self , n_jobs ) : 
 ---> 81 max_num_concurrent_tasks = self . _get_max_num_concurrent_tasks ( ) 
 82 if n_jobs is None : 
 83 n_jobs = 1 

 /databricks/python/lib/python3.7/site-packages/joblibspark/backend.py in _get_max_num_concurrent_tasks (self) 
 96 # maxNumConcurrentTasks() is a package private API 
 97 # pylint: disable=W0212 
 ---> 98 return self . _spark . sparkContext . _jsc . sc ( ) . maxNumConcurrentTasks ( ) 
 99 
 100 def abort_everything ( self , ensure_ready = True ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 330 raise Py4JError(
 331 "An er

In [24]:
import sklearn
sklearn.__version__

Out[3]: '0.23.2'